In [ ]:
import panel as pn
import panel.vega
import panel.plotly

pn.extension()

``Panel`` makes it possible to display and arrange a wide range of plots and other media on a page, including plots (matplotlib, bokeh, vega/altair, holoviews, and plotly), images (pngs, svgs, gifs, jpegs),  and text (Markdown, HTML etc.).

The ``pn.panel`` helper function returns a viewable representation of any object, e.g. a simple string will be interpreted as ``Markdown`` by default:

In [ ]:
layout = pn.panel('**Some text**')
layout

The ``pn.panel`` will always return a Panel layout containing one or more views representing the supplied object. However we can also access the items of the ``Panel``, e.g. to confirm the type of the ``Pane`` that was created:

In [ ]:
print(layout[0])

This also makes it easy to manually compose the visible representation of an object. However in some cases it is necessary to manually choose the appropriate representation of an object, which we can do by manually constructing the desired ``Pane``, e.g. we may want to render the string from above without rendering the Markdown syntax. We can do so by manually constructing a ``pn.pane.Str``:

In [ ]:
pane = pn.pane.Str('*Some text*')
pane

This also makes it easier to get a handle on the ``Pane`` object to update it. Since ``Pane`` objects are reactive updating the ``Pane.object`` will cause all existing views of the ``Pane`` to update in response:

In [ ]:
pane.object = 'Some updated text'

One thing to note is that since a ``Pane`` may define multiple views, when working with an explicitly constructed ``Pane`` care should be taken to compose the components of the ``Pane.layout`` manually.

### Bokeh

Since panel is built on top of bokeh it natively understand bokeh plots and models, which means we can easily mix and match bokeh and panel code:

In [ ]:
from bokeh.plotting import figure

fig = figure(width=300, height=300)
r = fig.line([1, 2, 3], [1, 2, 3])

bk_layout = pn.panel(fig)
bk_layout

To update the plot we can modify the bokeh model and ``trigger`` an update to the model parameter:

In [ ]:
r.data_source.data['y'] = r.data_source.data['y'][::-1]
bk_layout[0].param.trigger('object')

This will efficiently sync just the changes we made to the model. Alternatively we can also replace the model entirely:

In [ ]:
new_fig = figure(width=300, height=300)
r = new_fig.scatter([1, 2, 3], [1, 2, 3])

bk_layout[0].object = new_fig

### HoloViews

HoloViews often provides a more concise way of declaring plots and HoloViews objects are also supported natively by panel.

In [ ]:
import numpy as np
import holoviews as hv

box = hv.BoxWhisker((np.random.randint(0, 10, 100), np.random.randn(100)), 'Group').sort()

hv_layout = pn.panel(box)
hv_layout

A HoloViews panel can be updated in the same way as a matplotlib one:

In [ ]:
hv_layout[0].object = hv.Violin(box)

Additionally, since HoloViews is designed to make it easy to explore parameter spaces, the HoloViews pane will automatically add widgets when given a HoloMap or DynamicMap type. We can either display the default widget layout, or manually lay out the items in the layout:

In [ ]:
xs = np.linspace(0, np.pi*2)
hmap = hv.HoloMap({ph: hv.Curve((xs, np.sin(xs+ph))) for ph in np.linspace(0, np.pi*2, 11)}, 'Phase')

widget_layout = pn.panel(hmap)

pn.Column(widget_layout[1], widget_layout[0])

### Matplotlib

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111)
artist = ax.plot([1, 2, 3])[0]

mpl_layout = pn.panel(fig)
mpl_layout

In [ ]:
artist.set_ydata(artist.get_ydata()[::-1])
mpl_layout[0].param.trigger('object')

### Altair/Vega

Panel also provides a ``vega`` and ``vega-lite`` renderer which makes it possible to render ``altair`` plots. In addition to basic rendering the vega renderer will provide binary serialization for any array data sent to the browser, providing huge speedups over the standard JSON serialization employed by vega. This makes it possible to plot and interactively update much larger datasets.

In [ ]:
import altair as alt

from vega_datasets import data

cars = data.cars()

chart = alt.Chart(cars).mark_circle(size=60).encode(
    x='Horsepower',
    y='Miles_per_Gallon',
    color='Origin',
    tooltip=['Name', 'Origin', 'Horsepower', 'Miles_per_Gallon']
).interactive()

vega_layout = pn.panel(chart)
vega_layout

As for all other ``Pane`` objects, we can update the chart by setting the ``object``. In case of altair/vega ``Panel`` will even cache the data, avoiding sending the data needlessly:

In [ ]:
vega_layout[0].object = chart.mark_circle(size=100)

### Plotly Plots

In [ ]:
import numpy as np
import plotly.graph_objs as go

xx=np.linspace(-3.5, 3.5, 100)
yy=np.linspace(-3.5, 3.5, 100)
x,y=np.meshgrid(xx, yy)
z=np.exp(-(x-1)**2-y**2)-(x**3+y**4-x/5)*np.exp(-(x**2+y**2))

surface = go.Surface(z=z)
layout = go.Layout(
    title='Plotly 3D Plot',
    autosize=False,
    width=500,
    height=500,
    margin=dict(t=50, b=50, r=50, l=50)
)
fig = go.Figure(data=[surface], layout=layout)

plotly_layout = pn.panel(fig)
plotly_layout